In [1]:
import requests
import time as t
from mpl_toolkits.basemap import Basemap
from shapely.geometry import LineString
from matplotlib import pyplot as plt
import numpy as np
import os
import urllib
import json
%matplotlib inline

# Env vars and functions

In [2]:
def decode(encoded):
    inv = 1.0 / 1e6
    decoded = []
    previous = [0,0]
    i = 0
    while i < len(encoded):
        ll = [0,0]
        for j in [0, 1]:
            shift = 0
            byte = 0x20
            while byte >= 0x20:
                byte = ord(encoded[i]) - 63
                i += 1
                ll[j] |= (byte & 0x1f) << shift
                shift += 5
            ll[j] = previous[j] + (~(ll[j] >> 1) if ll[j] & 1 else (ll[j] >> 1))
            previous[j] = ll[j]
        decoded.append([float('%.6f' % (ll[1] * inv)), float('%.6f' % (ll[0] * inv))])
    return decoded


def synthesize_gps(edges, shape, distribution="normal", stddev=None,uuid='999999'):
    
    jsonDict = {"uuid":"100343", "trace":[]}
    coords = decode(shape)
    sttm = t.time() - 100000
    
    for i, edge in enumerate(edges):
        dist = edge['length']
        speed = edge['speed']
        beginShapeIndex = edge['begin_shape_index']
        endShapeIndex = edge['end_shape_index']
        lon, lat = coords[endShapeIndex]
        if stddev is not None:
            avgLat = np.mean(np.array(coords)[:,1])
            stddevLon = stddev / 111.111     # approx. 111.111 km per deg lon unless very close to the poles
            stddevLat = stddev / (111.111 * np.cos(avgLat))     # approx 111.111 km * cos(lat) per deg lat
            lon += np.random.normal(scale=stddevLon)
            lat += np.random.normal(scale=stddevLat)
        dur = dist / speed * 3600.0
        time = sttm + dur
        time = int(round(time))
        if i == 0:
            st_lon, st_lat = coords[beginShapeIndex]
            jsonDict["trace"].append({"lat":st_lat,"lon":st_lon,"time":time})
        jsonDict["trace"].append({"lat": lat,"lon":lon,"time":time})
        sttm = time

    return jsonDict

# Open Traffic Reporter Validation Service 

### 1. Get New Route

In [3]:
# # 349 38th
# stLat = 37.8261
# stLon = -122.25872

# 2527 24th 
stLat = 37.75286
stLon = -122.40486

endLat = 37.790168
endLon = -122.402264

jsonDict = {"locations":[
        {"lat":stLat,"lon":stLon,"type":"break"},
        {"lat":endLat,"lon":endLon,"type":"break"}],
            "costing":"auto",
            "id":"my_work_route"}

#### from mapzen.com 

In [5]:
# baseUrl = 'https://valhalla.mapzen.com/route'
# apiKey = os.environ.get('MAPZEN_API')
# payload = {"json": json.dumps(jsonDict), "apiKey": apiKey}
# route = requests.get(baseUrl, params=payload)

#### from local valhalla server (via docker)

In [5]:
baseUrl = 'http://localhost:8002/route?json='
route = requests.get(baseUrl + json.dumps(jsonDict))

In [8]:
print route.status_code
shape = route.json()['trip']['legs'][0]['shape']

200


### 2. Pass Route Shape to Map Matching API 

In [9]:
payload = {
    "encoded_polyline": shape,
    "costing": "pedestrian",
    "directions_options": {
        "units": "kilometers"
    },
    "shape_match": "map_snap",
    "trace_options": {
        "turn_penalty_factor": 500
    }
}

#### from mapzen.com

In [59]:
# baseUrl = 'https://valhalla.mapzen.com/trace_attributes?'
# apiKey = os.environ.get('MAPZEN_API')
# baseUrl += 'api_key=' + apiKey

#### from local valhalla server

In [10]:
baseUrl = 'http://valhalla:8002/trace_attributes?'

In [11]:
matched = requests.post(baseUrl, json=payload)

In [12]:
print 'Matched {0} points. Showing the first below: \n{1}'.format(
    len(matched.json()['matched_points']), matched.json()['matched_points'][0])

Matched 160 points. Showing the first below: 
{u'lon': -122.404884, u'distance_from_trace_point': 0.0, u'distance_along_edge': 0.52, u'type': u'matched', u'lat': 37.753098, u'edge_index': 0}


In [13]:
edges = matched.json()['edges']
print 'Got {0} edges. Showing the first below: \n {1}'.format(len(edges), edges[0])

Got 93 edges. Showing the first below: 
 {u'drive_on_right': True, u'max_downward_grade': 32768, u'end_heading': 265, u'surface': u'paved_smooth', u'names': [u'24th Street'], u'end_shape_index': 1, u'way_id': 24444478, u'speed': 35, u'id': 775381703218, u'pedestrian_type': u'foot', u'use': u'road', u'density': 14, u'begin_shape_index': 0, u'max_upward_grade': 32768, u'traversability': u'both', u'lane_count': 0, u'travel_mode': u'pedestrian', u'mean_elevation': 32768, u'weighted_grade': 0.0, u'end_node': {u'intersecting_edges': [{u'to_edge_name_consistency': False, u'walkability': u'both', u'begin_heading': 356, u'driveability': u'both', u'from_edge_name_consistency': False, u'cyclability': u'both'}, {u'to_edge_name_consistency': False, u'walkability': u'both', u'begin_heading': 175, u'driveability': u'both', u'from_edge_name_consistency': False, u'cyclability': u'both'}], u'elapsed_time': 29, u'type': u'street_intersection', u'time_zone': u'PST-08PDT+01,M3.2.0/02:00,M11.1.0/02:00', u'a

### 3. Create Fake GPS traces

#### Extract coords from routes.shape, perturb, and save to map-matched json

In [14]:
jsonDict = synthesize_gps(edges, shape, stddev=0.05)   # stddev in km

In [15]:
print 'Sending {0} fake GPS measurements to the reporter'.format(len(jsonDict['trace']))

Sending 94 fake GPS measurements to the reporter


### 4. Send Fake GPS to Reporter

In [16]:
baseUrl = 'http://reporter:8003/report?json='
report = requests.get(baseUrl + json.dumps(jsonDict))

In [17]:
segments = report.json()['segments']
print 'Reporter matched {0} GPS measurements to {1} OSMLR segments. Showing the first below: \n{2}'.format(
    len(jsonDict['trace']),len(segments), segments[25])

Reporter matched 94 GPS measurements to 45 OSMLR segments. Showing the first below: 
{u'segment_id': 190220441225, u'begin_shape_index': 59, u'start_time': 1495571978.0, u'length': 192, u'end_time': 1495571979.721, u'end_shape_index': 59}
